# "Eigenfactor" generation

In this notebook, I read in data from PubMed about articles and their corresponding citations. With this information, I create an adjacency matrix $(4000\times4000)$ tracking citations between *journals* in an attempt to identify which are the most influential. This is similar to the classic PageRank algorithm, with modifications/extensions proposed by Jevin West and others.

In [3]:
import numpy as np
import pandas as pd
from collections import OrderedDict
# from scipy import sparse as sp #sparse matrices are slower at this level of complexity (~4000**2 elems)

In [4]:
ALPHA = 0.85 # alpha = likelihood of staying, 1 - alpha = likelihood of randomly jumping / "teleporting".
EPSILON = 0.00001

## Data loading section

Here we read in the text data and process it so that it can be easily converted into an adjacency matrix. Null journal names are dropped because the name of the journal is used as a unique identifier later on.

In [5]:
%%time
links = pd.read_table('pubmed_links.txt.gz', sep="\t", dtype={'cited':int,'citing':int})
nodes = pd.read_table('pubmed_nodes.txt.gz', sep="\t", quotechar='"', quoting=3, lineterminator="\n", dtype={'journal': str, 'pmid':int, 'pmcid':str, 'title':str, 'date_pub':str})

CPU times: user 21.6 s, sys: 2.86 s, total: 24.5 s
Wall time: 25.6 s


In [6]:
#basically this is dropping extra cloumns from nodes, this could have been done in place but this was easier
#also dropping NAN's/nulls whenever they appear. this is primarily to eliminate null journal names which mess up the logic later on
nodes2 = pd.concat([nodes['pmid'], nodes['journal']], axis = 1, keys=['pmid', 'journal']).dropna(axis = 0, how='any', subset=('journal',))

In [7]:
%%time
#this is like a double join to get the journal info about the citing and cited journals, then extra columns are dropped
links_merged = links.merge(nodes2, left_on="cited", right_on="pmid", suffixes=("","_cited"))
links_merged = links_merged.merge(nodes2, left_on="citing", right_on="pmid", suffixes=("_cited","_citing"))
links_merged = links_merged[['citing', 'cited', 'journal_cited', 'journal_citing']]

CPU times: user 5.38 s, sys: 1.72 s, total: 7.1 s
Wall time: 8.45 s


In [8]:
# get unique journals and counts for articles in each journal
# the below is a *bit* redundant, but there might be a journal that is cited, but doesnt cite and vice versa
journals = np.unique(np.append(links_merged['journal_citing'].unique(), links_merged['journal_cited'].unique()))
art_counts = links_merged['journal_citing'].value_counts()

## Adjacency matrix

This is the most expensive part of the algorithm. The adjacency matrix $(\textbf{Z})$ is constructed citation by citation. The *citing* journal determines the column of the entry, and the *cited* journal determines the row. This is slightly different from most implementations, which are primarily row-based.

In [9]:
# prep for constructing the matrix
citations = links_merged.as_matrix(['journal_cited', 'journal_citing'])
Z = np.zeros((len(journals), len(journals)))

In [10]:
%%time
# fairly straight forward: for each citation we get the corresponding index for the citing and cited journal and then 
# increment the value in the matrix at those coordinates
for i, (cited, citing) in enumerate(citations):
    row = np.where(journals==cited)[0][0]
    col = np.where(journals==citing)[0][0]
    Z[row, col] += 1
    if i % 100000 == 0:
        print('Iteration number: %s' % i)

Iteration number: 0
Iteration number: 100000
Iteration number: 200000
Iteration number: 300000
Iteration number: 400000
Iteration number: 500000
Iteration number: 600000
Iteration number: 700000
Iteration number: 800000
Iteration number: 900000
Iteration number: 1000000
Iteration number: 1100000
Iteration number: 1200000
CPU times: user 4min 2s, sys: 2.05 s, total: 4min 4s
Wall time: 42min 47s


**Article vector**

This simply tracks how many articles each journal has, this is used to determine the "teleport" factor later on. This vector is later normalized.

In [11]:
# build the article vector
a = np.zeros((len(journals), 1))
for key, val in art_counts.items():
    i = np.where(journals==key)[0][0]
    a[i, 0] = val

In [12]:
# # to check against basic test matrix uncomment and run from this cell on
# a = np.array([[3], [2], [5], [1], [2], [1]], dtype=int)
# Z = np.zeros((6, 6)) # forcing ndarray, matrix object doesn't gel with column normalization later on
# Z[0,:] = [1, 0, 2, 0, 4, 3]
# Z[1,:] = [3, 0, 1, 1, 0, 0]
# Z[2,:] = [2, 0, 4, 0, 1, 0]
# Z[3,:] = [0, 0, 1, 0, 0, 1]
# Z[4,:] = [8, 0, 3, 0, 5, 2]
# Z[5,:] = [0, 0, 0, 0, 0, 0]
# # Z, a

In [13]:
np.fill_diagonal(Z, 0) #REMOVE self-citations

In [14]:
H = np.zeros_like(Z) #prep normalized adjacency matrix

**Normalize the adjacency matrix**

Here we create the normalized adjacency matrix $\textbf{H}$ by dividing each column of $\textbf{Z}$ by its sum. Anytime the sum is $0$, we note that in the vector $\vec{d}$. A column with sum $0$ is a dangling node (this journal cites no others! could also be called a trap) and we want to keep track of that for use later on in the iterative method.

In [15]:
%%time
# normalize columns and track dangling nodes
# could do this in one line with broadcasting but it seems to run into problems w/ divide by zeros
d = np.zeros((1, H.shape[0]), dtype=int)
for i, val in enumerate(Z.sum(axis=0).tolist()):
    if val:
        H[:,i] = Z[:,i] / val
    else:
        d[0,i] = 1

CPU times: user 611 ms, sys: 249 ms, total: 859 ms
Wall time: 1.27 s


**Normalize the article vector. Initialize and normalize the influence vector ($\pi$).** 

In [16]:
a = a / sum(a) #normalize
pi = np.ones((H.shape[0], 1)) / H.shape[0] #normalize

## Main iterative method

What we are really looking for is the leading eigenvector of $\textbf{P}$ where:

$\textbf{P} = \alpha \textbf{H'} + (1-\alpha) \vec{a} * \vec{e}^T$

with $\textbf{H'}$ equivalent to $\textbf{H}$ with all dangling nodes replaced by the article vector.

This dense matrix calculation is expensive, however, and the iterative method defined below will yield the leading eigenvector of $\textbf{P}$ much more quickly.


We iterate the following equation until it converges (defined as $\|\vec{\pi}_{k+1} - \vec{\pi}_k\| < \epsilon$) and yields the leading eigenvector we'll denote as $\vec{\pi}^*$:

$\vec{\pi}_{k+1} = \alpha * \textbf{H} * \vec{\pi}_k + (\alpha*\vec{d}*\vec{\pi}_k+(1-\alpha))*\vec{a}$

Where $\vec{d}$ is the vector tracking dangling nodes, $\vec{a}$ is the article vector, and $\alpha$/$\epsilon$ are defined above.

$\vec{\pi}$ could be any vector, but we initialize it s.t. all entires are equal and it is normalized. Because of the teleport term above, we are guaranteed convergence by the Perron-Frobenius Theorem ($\textbf{P}$ is an irreducible aperiodic Markov chain).

In [17]:
%%time
# now we just do our iterative method.
for i in range(0, 10000):
    pi_kpo = ALPHA * H @ pi + (ALPHA * (d @ pi) + (1 - ALPHA)).item(0) * a
    if np.linalg.norm(pi_kpo - pi, ord = 1) < EPSILON: #checking one-norm
        print('Ending after', i, 'iterations.')
        break
    else:
        pi = pi_kpo

Ending after 21 iterations.
CPU times: user 1.14 s, sys: 597 ms, total: 1.74 s
Wall time: 1.74 s


## The Eigenfactors

Here, we iterate the algorithm one last time on just the matrix $\textbf{H}$ (without the teleport term) to wash out all the weight applied to non-citing journals. Once we have done that, we normalize the vector and multiply by $100$ to create percentage **Eigenfactor** scores. This can be thought of as an influence score, or a probability describing how likely you are to end up at a given journal by following citations randomly. We are given a vector of these scores by:

$\vec{EF} = \frac{\textbf{H}*\vec{\pi}^*}{\|\textbf{H}*\vec{\pi}^*\|_1}$ 

The bottom term here is simply the sum of all elements in the top term, since we are guarnateed non-negative entries, however, the one-norm is equivalent. 

In [18]:
top_term = H @ pi_kpo
ef = 100 * (top_term / top_term.sum(0))

Here, we just do a little analysis and print out the journals, their rankings, and their corresponding scores.

In [19]:
#a little analysis, which is totally wrong when doing the basic matrix
scores = {}
for i, sc in enumerate(ef):
    scores[sc.item(0)] = journals[i]
    
ranks = OrderedDict(sorted(scores.items(), key=lambda t: t[0], reverse=True))
for i, (k, v) in enumerate(ranks.items()):
    print('Rank %s: %s, with score: %s' % ((i+1), v, k))

Rank 1: The Journal of Cell Biology, with score: 13.224570734402405
Rank 2: The Journal of Experimental Medicine, with score: 10.477527420941772
Rank 3: Nucleic Acids Research, with score: 7.430654663570389
Rank 4: PLoS ONE, with score: 5.028224520886794
Rank 5: Genome Biology, with score: 3.4306551538777668
Rank 6: PLoS Biology, with score: 3.280847940277107
Rank 7: British Journal of Cancer, with score: 3.27993106557557
Rank 8: Nature, with score: 2.609851769536778
Rank 9: BMC Bioinformatics, with score: 2.548252990114295
Rank 10: PLoS Genetics, with score: 2.134982431488588
Rank 11: BMC Genomics, with score: 1.7299694692157868
Rank 12: Emerging Infectious Diseases, with score: 1.563078694430931
Rank 13: The Journal of General Physiology, with score: 1.2340409949724136
Rank 14: Environmental Health Perspectives, with score: 1.23192130585293
Rank 15: PLoS Pathogens, with score: 1.187246392975638
Rank 16: PLoS Computational Biology, with score: 1.1258001278722436
Rank 17: PLoS Medicine

In [20]:
#see end results, kind of
pi_kpo, ef, Z, d, a

(array([[  1.73495475e-06],
        [  1.58744722e-06],
        [  1.01038185e-04],
        ..., 
        [  2.21471194e-05],
        [  1.91078999e-05],
        [  1.20236193e-03]]), array([[ 0.00015978],
        [ 0.00012677],
        [ 0.00832572],
        ..., 
        [ 0.00246201],
        [ 0.0020389 ],
        [ 0.09689764]]), array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  1.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]]), array([[0, 0, 0, ..., 0, 0, 0]]), array([[  2.46248810e-06],
        [  3.28331746e-06],
        [  1.95357389e-04],
        ..., 
        [  9.02912303e-06],
        [  1.23124405e-05],
        [  2.44032571e-03]]))